# 家計調査サンキーダイアグラム

準備として、ライブラリをインポートします。

In [ ]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
from pathlib import Path
# データ取得に用いるライブラリ
import requests
# データ処理に用いるライブラリ
import datetime
import pandas as pd
# 可視化に用いるライブラリ
import plotly
import plotly.graph_objs as go

In [ ]:
os.getcwd(), os.path.abspath(os.path.join(os.getcwd(), "../../.."))

In [ ]:
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [ ]:
from estat import (
    get_metainfo,
    get_statsdata,
    cleansing_statsdata,
    colname_to_japanese,
    create_hierarchy_dataframe
)

In [ ]:
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

## 家計調査のデータ取得と前処理

In [ ]:
annual_statsDataId = "0002070011"
annual_meta = get_metainfo(appId, annual_statsDataId)
annual_metadata = annual_meta["GET_META_INFO"]["METADATA_INF"]
annual_total_num = annual_metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
annual_total_num

In [ ]:
annual_metadata['CLASS_INF']['CLASS_OBJ'][0]

In [ ]:
annual_metadata['CLASS_INF']['CLASS_OBJ'][1]['CLASS'][:5]

In [ ]:
annual_metadata['CLASS_INF']['CLASS_OBJ'][2]

In [ ]:
annual_metadata['CLASS_INF']['CLASS_OBJ'][3]['CLASS'][:5]

In [ ]:
annual_metadata['CLASS_INF']['CLASS_OBJ'][4]

In [ ]:
annual_metadata['CLASS_INF']['CLASS_OBJ'][5]["@name"]

In [ ]:
annual_metadata['CLASS_INF']['CLASS_OBJ'][5]['CLASS'][:5]

In [ ]:
annual_params = {
    "cdCat02": "04",  # 二人以上の世帯のうち勤労者世帯（2000年～）で絞る
    "cdCat03": "208",  # 世帯主の年齢階級を40～44歳で絞る
    "cdTime": "2024000000",  # 時間軸を2024で絞る
}
annual_data = get_statsdata(appId, annual_statsDataId, params=annual_params)
annual_value = colname_to_japanese(cleansing_statsdata(annual_data))

In [ ]:
annual_value.columns

In [ ]:
cond = annual_value["単位"] == "円"
cond &= annual_value['用途分類階層レベル'].astype(int) <= 4
# 可処分所得233、黒字234を除外
cond &= ~annual_value["用途分類コード"].isin(["233", "234"])
cond &= ~annual_value["用途分類"].str.contains("再掲")
cols = ["用途分類コード", "用途分類", '用途分類階層レベル', "値"]
annual_df = annual_value.loc[cond, cols]
annual_df.head()

## サンキーダイアグラム描画

In [ ]:
def create_code_mappings(annual_metadata):
    """
    メタデータからコードマッピング辞書を作成
    
    Parameters
    ----------
    annual_metadata : dict
        メタデータ辞書
    
    Returns
    -------
    code_to_name : dict
        コード->名前のマッピング辞書
    code_to_parent : dict
        コード->親コードのマッピング辞書
    name_to_code : dict
        名前->コードのマッピング辞書
    """
    class_list = annual_metadata['CLASS_INF']['CLASS_OBJ'][1]['CLASS']
    
    code_to_name = {}
    code_to_parent = {}
    
    for item in class_list:
        if item.get('@unit') == '円':
            code = item['@code']
            name = item['@name']
            code_to_name[code] = name
            parent = item.get('@parentCode')
            if parent and parent != '':
                code_to_parent[code] = parent
    
    # 逆マッピングを作成
    name_to_code = {name: code for code, name in code_to_name.items()}
    
    return code_to_name, code_to_parent, name_to_code

In [ ]:
code_to_name, code_to_parent, name_to_code = create_code_mappings(annual_metadata)

In [ ]:
[item for item in code_to_name.items()][:5]

In [ ]:
def get_root_parent(code, code_to_parent):
    """
    コードのルート親を取得（018または057）
    
    Parameters
    ----------
    code : str
        対象コード
    code_to_parent : dict
        親コードのマッピング辞書
    
    Returns
    -------
    str or None
        ルート親コード（'018'または'057'）。見つからない場合はNone
    """
    current = code
    visited = set()  # 無限ループ防止
    while current in code_to_parent:
        if current in visited:
            break
        visited.add(current)
        parent = code_to_parent[current]
        if parent in ['018', '057']:
            return parent
        current = parent
    return None

In [ ]:
get_root_parent('026', code_to_parent)

In [ ]:
def create_sankey_links(annual_df, code_to_name, code_to_parent, code_to_value):
    """
    サンキーダイアグラム用のリンクデータを作成
    
    Parameters
    ----------
    annual_df : pd.DataFrame
        年次データのDataFrame
    code_to_name : dict
        コード->名前のマッピング辞書
    code_to_parent : dict
        コード->親コードのマッピング辞書
    code_to_value : dict
        コード->値のマッピング辞書
    
    Returns
    -------
    pd.DataFrame
        リンク情報を含むDataFrame（source, target, value, source_code, target_codeカラム）
    """
    links = []
    
    # 各コードについて親との関係を作成
    for code in annual_df['用途分類コード']:
        code_str = str(code)
        if code_str in code_to_parent and code_str not in ['018', '057']:
            parent_code = code_to_parent[code_str]
            value = code_to_value.get(code, 0)
            
            if value > 0:
                child_name = code_to_name.get(code, str(code))
                parent_name = code_to_name.get(parent_code, str(parent_code))
                
                root = get_root_parent(code_str, code_to_parent)
                
                if root == '018':
                    # 受取側: child -> parent
                    links.append({
                        'source': child_name,
                        'target': parent_name,
                        'value': value,
                        'source_code': code,
                        'target_code': parent_code
                    })
                elif root == '057':
                    # 支払側: parent -> child
                    links.append({
                        'source': parent_name,
                        'target': child_name,
                        'value': value,
                        'source_code': parent_code,
                        'target_code': code
                    })
    
    # 受取から支払への接続（中央）
    receive_value = code_to_value.get('018', 0)
    if receive_value > 0:
        receive_name = code_to_name.get('018', '受取')
        payment_name = code_to_name.get('057', '支払')
        
        links.append({
            'source': receive_name,
            'target': payment_name,
            'value': receive_value,
            'source_code': '018',
            'target_code': '057'
        })
    
    return pd.DataFrame(links)

In [ ]:
code_to_value = dict(zip(annual_df['用途分類コード'], annual_df['値']))
[item for item in code_to_value.items()][:5]

In [ ]:
sankey_df = create_sankey_links(annual_df, code_to_name, code_to_parent, code_to_value)
sankey_df.head()

In [ ]:
def assign_node_colors(all_nodes, name_to_code, code_to_parent):
    """
    サンキーダイアグラムのノード色を設定
    
    Parameters
    ----------
    all_nodes : list
        全ノードのリスト
    name_to_code : dict
        ノード名->コードのマッピング辞書
    code_to_parent : dict
        コード->親コードのマッピング辞書
    
    Returns
    -------
    list
        ノードごとの色のリスト（RGBA形式の文字列）
    """
    node_colors = []
    for node in all_nodes:
        code = name_to_code.get(node)
        
        if code == '018':
            node_colors.append('rgba(100, 150, 250, 0.9)')  # 受取（濃い青）
        elif code == '057':
            node_colors.append('rgba(250, 100, 100, 0.9)')  # 支払（濃い赤）
        else:
            root = get_root_parent(code, code_to_parent)
            if root == '018':
                node_colors.append('rgba(135, 206, 250, 0.7)')  # 受取側（水色）
            elif root == '057':
                node_colors.append('rgba(255, 160, 122, 0.7)')  # 支払側（サーモン）
            else:
                node_colors.append('rgba(200, 200, 200, 0.7)')
    
    return node_colors

In [ ]:
all_nodes = list(set(sankey_df['source'].tolist() + sankey_df['target'].tolist()))
all_nodes[:5]

In [ ]:
node_colors = assign_node_colors(all_nodes, node_to_code, code_to_parent)
node_colors[:5]

In [ ]:
def create_plotly_sankey_figure(sankey_df, node_colors, title=None, 
                                source_name=None, source_org=None, source_url=None,
                                annotation_x=0.5, annotation_y=1.0,
                                xanchor="center", yanchor="top",
                                annotation_font_size=12, annotation_font_color="gray"):
    """
    Plotlyサンキーダイアグラムを作成
    
    Parameters
    ----------
    sankey_df : pd.DataFrame
        リンク情報を含むDataFrame（source, target, valueカラムを含む）
    node_colors : list
        ノードごとの色のリスト
    title : str, optional
        グラフのタイトル
    source_name : str, optional
        データソース名（例: "家計調査結果"）
    source_org : str, optional
        データソース提供組織（例: "総務省統計局"）
    source_url : str, optional
        データソースURL（例: "http://www.stat.go.jp/data/kakei/index.htm"）
    annotation_x : float, optional
        アノテーションのx位置
    annotation_y : float, optional
        アノテーションのy位置
    annotation_font_size : int, optional
        アノテーションのフォントサイズ
    annotation_font_color : str, optional
        アノテーションのフォントカラー（デフォルト: "gray"）
    
    Returns
    -------
    go.Figure
        Plotlyのサンキーダイアグラム
    """
    # ノードのリストを作成
    all_nodes = list(set(sankey_df['source'].tolist() + sankey_df['target'].tolist()))
    node_to_idx = {node: idx for idx, node in enumerate(all_nodes)}
    
    # インデックスに変換
    source_idx = sankey_df['source'].map(node_to_idx).tolist()
    target_idx = sankey_df['target'].map(node_to_idx).tolist()
    values = sankey_df['value'].tolist()
    
    # サンキーダイアグラムの作成
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=20,
            thickness=15,
            line=dict(color="white", width=1),
            label=all_nodes,
            color=node_colors,
            hovertemplate='%{label}<br>値: %{value:,.0f}円<extra></extra>'
        ),
        link=dict(
            source=source_idx,
            target=target_idx,
            value=values,
            color='rgba(200, 200, 200, 0.3)',
            hovertemplate='%{source.label} -> %{target.label}<br>金額: %{value:,.0f}円<extra></extra>'
        )
    )])
    
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(size=16)
        ),
        font_size=10,
        height=900,
        width=1400,
        plot_bgcolor='white',
        paper_bgcolor='white'
    )
    
    # 出典情報を生成してアノテーションを追加
    if source_name and source_org and source_url:
        today = datetime.date.today()
        date_str = f"{today.year}年{today.month}月{today.day}日"
        annotation_text = f"＊「{source_name}」（{source_org}）（{source_url}）（{date_str}に利用）"
        
        fig.add_annotation(
            text=annotation_text,
            xref="paper",
            yref="paper",
            x=annotation_x, 
            y=annotation_y, 
            xanchor=xanchor,
            yanchor=yanchor,
            showarrow=False,
            font={"size": annotation_font_size, "color": annotation_font_color}
        )
    
    return fig

In [ ]:
title = "家計調査 二人以上の世帯のうち勤労者世帯（2000年～）・2024年・世帯主の年齢階級を40～44歳：受取・支払サンキーダイアグラム"
source_name="家計調査結果"
source_org="総務省統計局"
source_url="http://www.stat.go.jp/data/kakei/index.htm"

In [ ]:
def create_sankey_diagram(annual_df, annual_metadata, title=None, 
                         source_name=None, source_org=None, source_url=None,
                         annotation_x=0.5, annotation_y=1.0,
                         xanchor="center", yanchor="top",
                         annotation_font_size=12, annotation_font_color="gray"):
    """
    家計調査データからサンキーダイアグラムを作成
    
    受取側: level4 -> level3 -> level2 -> level1 -> 受取(018)
    中央: 受取(018) -> 支払(057)
    支払側: 支払(057) -> level1 -> level2 -> level3 -> level4
    
    Parameters
    ----------
    annual_df : pd.DataFrame
        年次データのDataFrame
    annual_metadata : dict
        メタデータ辞書
    title : str, optional
        グラフのタイトル
    source_name : str, optional
        データソース名（例: "家計調査結果"）
    source_org : str, optional
        データソース提供組織（例: "総務省統計局"）
    source_url : str, optional
        データソースURL（例: "http://www.stat.go.jp/data/kakei/index.htm"）
    annotation_x : float, optional
        アノテーションのx位置
    annotation_y : float, optional
        アノテーションのy位置
    annotation_font_size : int, optional
        アノテーションのフォントサイズ
    annotation_font_color : str, optional
        アノテーションのフォントカラー（デフォルト: "gray"）
    
    Returns
    -------
    fig : go.Figure
        Plotlyのサンキーダイアグラム
    sankey_df : pd.DataFrame
        リンク情報を含むDataFrame
    """
    # 1. コードマッピングの作成
    code_to_name, code_to_parent, name_to_code = create_code_mappings(annual_metadata)
    
    # 2. DataFrameから値とレベルのマッピング
    code_to_value = dict(zip(annual_df['用途分類コード'], annual_df['値']))
    
    # 3. リンクデータの作成
    sankey_df = create_sankey_links(annual_df, code_to_name, code_to_parent, code_to_value)
    
    # 4. ノードリストの作成
    all_nodes = list(set(sankey_df['source'].tolist() + sankey_df['target'].tolist()))
    
    # 5. ノード色の設定
    node_colors = assign_node_colors(all_nodes, name_to_code, code_to_parent)
    
    # 6. グラフの作成
    fig = create_plotly_sankey_figure(
        sankey_df, 
        node_colors, 
        title,
        source_name,
        source_org,
        source_url,
        annotation_x,
        annotation_y,
        xanchor,
        yanchor,
        annotation_font_size,
        annotation_font_color
    )
    
    return fig, sankey_df

In [ ]:
fig, sankey_df = create_sankey_diagram(annual_df, annual_metadata,
    title=title,
    source_name=source_name,
    source_org=source_org,
    source_url=source_url,
    annotation_x=0.5,
    annotation_y=1.05,
    annotation_font_size=14,
)
fig.show()

In [ ]:
fig.write_html("kakei_sankey.html")

In [ ]:
# データの確認と検証
print(f"\n作成されたリンク数: {len(sankey_df)}")
print(f"ノード数: {len(set(sankey_df['source'].tolist() + sankey_df['target'].tolist()))}")

# 受取と支払の合計が一致するか確認
receive_total = sankey_df[sankey_df['target'] == '018_受取']['value'].sum()
payment_total = sankey_df[sankey_df['source'] == '057_支払']['value'].sum()
center_flow = sankey_df[(sankey_df['source'] == '018_受取') & (sankey_df['target'] == '057_支払')]['value'].sum()

print(f"\n受取への流入合計: {receive_total:,.0f}円")
print(f"支払からの流出合計: {payment_total:,.0f}円")
print(f"中央の流れ（受取 -> 支払）: {center_flow:,.0f}円")
print(f"バランスチェック: {'OK' if abs(receive_total - payment_total) < 1 else 'NG'}")

print("\nサンキーダイアグラム用データ（サンプル）:")
print(sankey_df[['source', 'target', 'value']].head(20))